## Курсовая Работа

### 1. Алгоритм в лоб

In [1]:
from datetime import datetime, timedelta

SHIFT_8 = timedelta(hours=8)
SHIFT_12 = timedelta(hours=12)
WEEKDAYS = ['Понедельник', 'Вторник', 'Среда', 'Четверг', 'Пятница']
WEEKENDS = ['Суббота', 'Воскресенье']

class Driver:
    def __init__(self, driver_id, shift_type):
        self.driver_id = driver_id
        self.shift_type = shift_type
        self.schedule = {day: [] for day in WEEKDAYS + WEEKENDS}
        self.breaks = {day: [] for day in WEEKDAYS + WEEKENDS}

class Bus:
    def __init__(self, bus_id, bus_index):
        self.bus_id = bus_id
        self.schedule = {day: [] for day in WEEKDAYS + WEEKENDS}
        self.drivers = []
        self.bus_index = bus_index

drivers_count = 0
drivers = []
buses = [Bus(f"Автобус {i+1}", i) for i in range(8)]

for driver in drivers:
    driver.schedule = {day: [] for day in WEEKDAYS + WEEKENDS}
    driver.breaks = {day: [] for day in WEEKDAYS + WEEKENDS}

for bus in buses:
    bus.schedule = {day: [] for day in WEEKDAYS + WEEKENDS}

def time_intervals(start_time, shift_duration):
    start = datetime.strptime(start_time, "%H:%M")
    end = start + shift_duration
    return start, end

def set_breaks(driver, start, end, break_type):
    breaks = []
    
    if break_type == "15_мин":
        current = start + timedelta(minutes=105)

        if driver.shift_type == "12_час":
            break_duration = timedelta(minutes=18)
            while current + break_duration < end:
                breaks.append((current.strftime("%H:%M"), (current + break_duration).strftime("%H:%M"), "15_мин"))
                current += timedelta(minutes=120)
        else:
            break_duration = timedelta(minutes=15)
            while current + break_duration < end:
                breaks.append((current.strftime("%H:%M"), (current + break_duration).strftime("%H:%M"), "15_мин"))
                current += timedelta(minutes=120)

    elif break_type == "1_час":
        if start.strftime("%H:%M") == "09:00":
            lunch_start = datetime.strptime("14:15", "%H:%M")
            lunch_end = lunch_start + timedelta(minutes=45)
        else:
            lunch_start = datetime.strptime("13:00", "%H:%M")
            lunch_end = lunch_start + timedelta(hours=1)

        if start <= lunch_start < end:
            breaks.append((lunch_start.strftime("%H:%M"), lunch_end.strftime("%H:%M"), "1_час"))
        elif lunch_start <= start < lunch_end:
            breaks.append((start.strftime("%H:%M"), (start + timedelta(hours=1)).strftime("%H:%M"), "1_час"))

    return breaks

def set_8_drivers():
    global drivers_count

    for bus_index, bus in enumerate(buses):
        if bus_index < 4:
            shift_start_time = "06:00"
        else:
            shift_start_time = "09:00"

        drivers_count += 1
        driver = Driver(f"Водитель {drivers_count} (8 час)", "8_час")
        drivers.append(driver)

        for day in WEEKDAYS:
            various_start_time = (datetime.strptime(shift_start_time, "%H:%M") + timedelta(minutes=(bus_index % 4) * 15)).strftime("%H:%M")
            start, end = time_intervals(various_start_time, SHIFT_8)

            if "09:00" in various_start_time:
                breaks = set_breaks(driver, start, end, "1_час")
            else:
                breaks = set_breaks(driver, start, end, "15_мин")

            driver.breaks[day] = breaks
            driver.schedule[day].append((start, end, bus.bus_id))
            bus.schedule[day].append((start, end, driver.driver_id))

def set_12_drivers():
    global drivers_count
    rest_12_counter = {}

    for bus_index, bus in enumerate(buses):
        if bus_index < 4:  
            for i in range(3):
                drivers_count += 1
                driver = Driver(f"Водитель {drivers_count} (12 час)", "12_час")
                bus.drivers.append(driver)
                drivers.append(driver)
                rest_12_counter[driver.driver_id] = 0
        else:
            for i in range(3):
                drivers_count += 1
                driver = Driver(f"Водитель {drivers_count} (12 час)", "12_час")
                bus.drivers.append(driver)
                drivers.append(driver)
                rest_12_counter[driver.driver_id] = 0

    for bus in buses:
        driver_index = 0

        for day in WEEKDAYS + WEEKENDS:
            while rest_12_counter[bus.drivers[driver_index].driver_id] > 0:
                rest_12_counter[bus.drivers[driver_index].driver_id] -= 1
                driver_index = (driver_index + 1) % len(bus.drivers)

            driver = bus.drivers[driver_index]

            if day in WEEKDAYS:
                if bus.bus_index < 4:
                    various_start_time = (datetime.strptime("14:00", "%H:%M") + timedelta(minutes=bus.bus_index * 15)).strftime("%H:%M")
                else:
                    various_start_time = (datetime.strptime("17:00", "%H:%M") + timedelta(minutes=(bus.bus_index - 4) * 15)).strftime("%H:%M")
            else:
                various_start_time = (datetime.strptime("06:00", "%H:%M") + timedelta(minutes=bus.bus_index * 18)).strftime("%H:%M")

            start, end = time_intervals(various_start_time, SHIFT_12)

            if day in WEEKDAYS + WEEKENDS:
                if start.hour == 6 and start.minute == 0:
                    breaks = set_breaks(driver, start, end, "1_час")
                else:
                    breaks = set_breaks(driver, start, end, "15_мин")
            else:
                breaks = set_breaks(driver, start, end, "15_мин")

            driver.breaks[day] = breaks
            driver.schedule[day].append((start, end, bus.bus_id))
            bus.schedule[day].append((start, end, driver.driver_id))

            rest_12_counter[driver.driver_id] = 2

            driver_index = (driver_index + 1) % len(bus.drivers)

def set_weekend_12_drivers():
    global drivers_count

    for _, bus in enumerate(buses):
        for day in WEEKENDS:
            drivers_count += 1
            driver = Driver(f"Водитель {drivers_count} (12 час)", f"12_час Выходной {day}")
            bus.drivers.append(driver)
            drivers.append(driver)

            start_time = (datetime.strptime("18:03", "%H:%M") + timedelta(minutes=bus.bus_index * 18)).strftime("%H:%M")
            start, end = time_intervals(start_time, SHIFT_12)

            breaks = set_breaks(driver, start, end, "15_мин")
            driver.breaks[day] = breaks
            driver.schedule[day].append((start, end, bus.bus_id))
            bus.schedule[day].append((start, end, driver.driver_id))

set_8_drivers()
set_12_drivers()
set_weekend_12_drivers()